In [1]:
import pandas as pd
import glob
import regex as re
import querygen as qg

In [2]:
files = glob.glob("results/*.csv")
dfs = [pd.read_csv(f, index_col=0) for f in files]
files

['results/eval_results_DBpedia_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_UniProt_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DBpedia_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_BTO_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_BTO_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_Yago_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_UniProt_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_UniProt_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_Yago_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_BTO_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_DBpedia_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_UniProt_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GG

In [3]:
dfs

[      f1_score  precision    recall  f1_score_rel  precision_rel  recall_rel  \
 0     0.000000   0.000000  0.000000      0.000000           0.00       0.000   
 1     0.000000   0.000000  0.000000      0.000000           0.00       0.000   
 2     0.666667   0.666667  0.666667      1.000000           1.00       1.000   
 3     0.000000   0.000000  0.000000      0.000000           0.00       0.000   
 4     0.000000   0.000000  0.000000      0.000000           0.00       0.000   
 ...        ...        ...       ...           ...            ...         ...   
 2380  0.000000   0.000000  0.000000      0.000000           0.00       0.000   
 2381  0.857143   0.857143  0.857143      0.166667           0.25       0.125   
 2382  0.000000   0.000000  0.000000      0.000000           0.00       0.000   
 2383  0.000000   0.000000  0.000000      0.000000           0.00       0.000   
 2384  0.615385   0.666667  0.571429      0.933333           1.00       0.875   
 
        ged  normed_ged   

In [4]:

def graph_from_erl(erl: qg.EnrichedEntitiesRelations):
    G = qg.nx.DiGraph()
    for node in erl.entities:
        G.add_node(node.identifier, label=node.type)
    for link in erl.relations:
        G.add_edge(
            link.entity,
            link.target,
            weight=link.link.instance_count,
            label=link.relation,
        )
    return G

In [5]:
for f, df in zip(files, dfs):
    # data=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    # prompt_type=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    model = re.findall(r"results/eval_results_\w+_(.+)\.csv", f)[0]
    # df['prompt_type'] = prompt_type
    df["model"] = (
        model.replace("_", " ")
        .replace("-", " ")
        .rstrip("-GGUF")
        .lstrip("NousResearch Hermes 3 Llama")
        .lstrip(".2")
        .lstrip(".3")
    )

df = pd.concat(dfs)
df

,f1_score,precision,recall,f1_score_rel,precision_rel,recall_rel,ged,normed_ged,response,model,cfg_name,zeroshot,stage,erl,generator,n_nodes,seed,erl_loaded
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0,0.000000,"A animal, born in nature, but died in a town",1 8B,DBpedia,True,raw,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3,60,"relations=[EnrichedRelation(entity='animal', r..."
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0,0.000000,"A animal, born in nature, but died in a town",1 8B,DBpedia,True,constrained,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3,60,"relations=[EnrichedRelation(entity='animal', r..."
2,0.666667,0.666667,0.666667,1.000000,1.000000,1.000000,1.0,0.800000,"A animal, born in nature, but died in a town",1 8B,DBpedia,True,aligned,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3,60,"relations=[EnrichedRelation(entity='animal', r..."
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0,0.000000,A step in the career of a musical artist,1 8B,DBpedia,True,raw,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3,12,relations=[EnrichedRelation(entity='musical ar...
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,0.000000,A step in the career of a musical artist,1 8B,DBpedia,True,constrained,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3,12,relations=[EnrichedRelation(entity='musical ar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.000000,a creative work has a in language with languag...,1 8B,Yago,True,constrained,"{""relations"":[{""entity"":""creative work"",""relat...",templated,7,126,relations=[EnrichedRelation(entity='creative w...
1130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.000000,a creative work has a in language with languag...,1 8B,Yago,True,aligned,"{""relations"":[{""entity"":""creative work"",""relat...",templated,7,126,relations=[EnrichedRelation(entity='creative w...
1131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.0,0.000000,"A location has geographical coordinates, a sch...",1 8B,Yago,True,raw,"{""relations"":[{""entity"":""location"",""relation"":...",llama,7,127,"relations=[EnrichedRelation(entity='location',..."
1132,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.0,0.000000,"A location has geographical coordinates, a sch...",1 8B,Yago,True,constrained,"{""relations"":[{""entity"":""location"",""relation"":...",llama,7,127,"relations=[EnrichedRelation(entity='location',..."


In [6]:
df

,f1_score,precision,recall,f1_score_rel,precision_rel,recall_rel,ged,normed_ged,response,model,cfg_name,zeroshot,stage,erl,generator,n_nodes,seed,erl_loaded
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0,0.000000,"A animal, born in nature, but died in a town",1 8B,DBpedia,True,raw,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3,60,"relations=[EnrichedRelation(entity='animal', r..."
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0,0.000000,"A animal, born in nature, but died in a town",1 8B,DBpedia,True,constrained,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3,60,"relations=[EnrichedRelation(entity='animal', r..."
2,0.666667,0.666667,0.666667,1.000000,1.000000,1.000000,1.0,0.800000,"A animal, born in nature, but died in a town",1 8B,DBpedia,True,aligned,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3,60,"relations=[EnrichedRelation(entity='animal', r..."
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0,0.000000,A step in the career of a musical artist,1 8B,DBpedia,True,raw,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3,12,relations=[EnrichedRelation(entity='musical ar...
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,0.000000,A step in the career of a musical artist,1 8B,DBpedia,True,constrained,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3,12,relations=[EnrichedRelation(entity='musical ar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.000000,a creative work has a in language with languag...,1 8B,Yago,True,constrained,"{""relations"":[{""entity"":""creative work"",""relat...",templated,7,126,relations=[EnrichedRelation(entity='creative w...
1130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.000000,a creative work has a in language with languag...,1 8B,Yago,True,aligned,"{""relations"":[{""entity"":""creative work"",""relat...",templated,7,126,relations=[EnrichedRelation(entity='creative w...
1131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.0,0.000000,"A location has geographical coordinates, a sch...",1 8B,Yago,True,raw,"{""relations"":[{""entity"":""location"",""relation"":...",llama,7,127,"relations=[EnrichedRelation(entity='location',..."
1132,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.0,0.000000,"A location has geographical coordinates, a sch...",1 8B,Yago,True,constrained,"{""relations"":[{""entity"":""location"",""relation"":...",llama,7,127,"relations=[EnrichedRelation(entity='location',..."


In [7]:
df["prompt_type"] = df["zeroshot"].apply(lambda x: "zero shot" if x else "one shot")
df.drop(columns=["zeroshot"], inplace=True)
df

,f1_score,precision,recall,f1_score_rel,precision_rel,recall_rel,ged,normed_ged,response,model,cfg_name,stage,erl,generator,n_nodes,seed,erl_loaded,prompt_type
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0,0.000000,"A animal, born in nature, but died in a town",1 8B,DBpedia,raw,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3,60,"relations=[EnrichedRelation(entity='animal', r...",zero shot
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0,0.000000,"A animal, born in nature, but died in a town",1 8B,DBpedia,constrained,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3,60,"relations=[EnrichedRelation(entity='animal', r...",zero shot
2,0.666667,0.666667,0.666667,1.000000,1.000000,1.000000,1.0,0.800000,"A animal, born in nature, but died in a town",1 8B,DBpedia,aligned,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3,60,"relations=[EnrichedRelation(entity='animal', r...",zero shot
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0,0.000000,A step in the career of a musical artist,1 8B,DBpedia,raw,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3,12,relations=[EnrichedRelation(entity='musical ar...,zero shot
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,0.000000,A step in the career of a musical artist,1 8B,DBpedia,constrained,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3,12,relations=[EnrichedRelation(entity='musical ar...,zero shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.000000,a creative work has a in language with languag...,1 8B,Yago,constrained,"{""relations"":[{""entity"":""creative work"",""relat...",templated,7,126,relations=[EnrichedRelation(entity='creative w...,zero shot
1130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.000000,a creative work has a in language with languag...,1 8B,Yago,aligned,"{""relations"":[{""entity"":""creative work"",""relat...",templated,7,126,relations=[EnrichedRelation(entity='creative w...,zero shot
1131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.0,0.000000,"A location has geographical coordinates, a sch...",1 8B,Yago,raw,"{""relations"":[{""entity"":""location"",""relation"":...",llama,7,127,"relations=[EnrichedRelation(entity='location',...",zero shot
1132,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.0,0.000000,"A location has geographical coordinates, a sch...",1 8B,Yago,constrained,"{""relations"":[{""entity"":""location"",""relation"":...",llama,7,127,"relations=[EnrichedRelation(entity='location',...",zero shot


In [8]:
df_clean = df.dropna()
df_clean.describe()

,f1_score,precision,recall,f1_score_rel,precision_rel,recall_rel,ged,normed_ged,n_nodes,seed
count,21405.000000,21405.000000,21405.000000,21405.000000,21405.000000,21405.000000,21405.000000,21405.000000,21405.000000,21405.000000
mean,0.250455,0.269640,0.243886,0.290617,0.302472,0.287232,8.814109,0.170970,4.034338,60.682691
std,0.362763,0.385082,0.360814,0.417230,0.430905,0.417090,24.995316,0.299057,1.863218,37.576947
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.384615,2.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,2.000000,28.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,3.000000,59.000000
75%,0.500000,0.500000,0.500000,0.666667,0.800000,0.666667,9.000000,0.333333,5.000000,93.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,409.000000,1.000000,10.000000,231.000000


In [9]:
df["erl_loaded"] = df["erl"].apply(qg.EnrichedEntitiesRelations.model_validate_json)
# df["response_loaded"] = df["response"].apply(lambda x: qg.EnrichedEntitiesRelations.model_validate_json(x) if isinstance(x, str) else None)
df["erl_graph"] = df["erl_loaded"].apply(graph_from_erl)
# df["response_graph"] = df["response"].apply(lambda x: graph_from_erl if x is not None else None)

In [10]:
# df["normed_ged"] = 1 - df["ged"] / df["erl_graph"].apply(
#     lambda x: (len(x.nodes) + len(x.nodes))
# )

In [11]:
df_clean["generator"].unique(), df_clean["model"].unique()

(array(['human', 'llama', 'templated'], dtype=object),
 array(['1 8B ', ' 3B '], dtype=object))

In [12]:
means = df_clean.groupby(
    ["n_nodes", "prompt_type", "stage", "model", "generator", "cfg_name"]
)[["f1_score", "normed_ged"]].mean()
counts = df_clean.groupby(
    ["n_nodes", "stage", "prompt_type", "model", "generator", "cfg_name"]
)[["f1_score", "normed_ged"]].count()
means

f1_score  normed_ged
n_nodes prompt_type stage       model generator cfg_name                      
2       one shot    aligned      3B   llama     BTO       0.472358    0.401760
                                                DBpedia   0.628616    0.468264
                                                UniProt   0.529004    0.520111
                                                Yago      0.636508    0.506325
                                      templated BTO       0.586957    0.417909
...                                                            ...         ...
10      zero shot   aligned     1 8B  human     DBpedia   0.558333    0.298571
                    constrained  3B   human     DBpedia   0.000000   -0.144528
                                1 8B  human     DBpedia   0.000000   -0.187619
                    raw          3B   human     DBpedia   0.000000   -0.100251
                                1 8B  human     DBpedia   0.279739   -0.020802

[405 rows x 2 columns]

In [13]:
counts

f1_score  normed_ged
n_nodes stage       prompt_type model generator cfg_name                      
2       aligned     one shot     3B   llama     BTO             41          41
                                                DBpedia        106         106
                                                UniProt         77          77
                                                Yago            63          63
                                      templated BTO             23          23
...                                                            ...         ...
10      constrained zero shot   1 8B  human     DBpedia          3           3
        raw         one shot     3B   human     DBpedia          3           3
                                1 8B  human     DBpedia          3           3
                    zero shot    3B   human     DBpedia          3           3
                                1 8B  human     DBpedia          3           3

[405 rows x 2 columns]

In [14]:
column_map = {
    "f1_score": "$F_1$",
    "normed_ged": "$GED_{s}$",
    "stage": "Stage",
    "prompt_type": "Prompt Type",
    "model": "Version \\& Size",
    "n_nodes": "$k$",
    "generator": "Query Origin",
}

## Compare Query Origins on DBpedia with 8B model

In [15]:
query_gen_compare = df_clean[
    df_clean["model"].str.contains("3B")
    & df_clean["cfg_name"].str.contains("DBpedia")
    & df_clean["prompt_type"].str.contains("one shot")
]
grouped_query_gen_compare = query_gen_compare.groupby(
    ["n_nodes", "generator", "stage"]
)[["f1_score", "normed_ged"]].mean()
grouped_query_gen_compare = grouped_query_gen_compare.rename(
    columns=column_map
).rename_axis([column_map["n_nodes"], column_map["generator"], column_map["stage"]], axis=0)
with open("tables/compare_gen_query.tex", "w") as f:
    f.write(
        grouped_query_gen_compare.to_latex(
            float_format="%.2f",
            caption="Comparison of query generation methods for LLama 3.1 8B on DBpedia with one shot prompts.",
            label="tab:compare_gen_query",
        )
    )
grouped_query_gen_compare

$F_1$  $GED_{s}$
$k$ Query Origin Stage                           
2   llama        aligned      0.628616   0.468264
                 constrained  0.000000  -0.001887
                 raw          0.629874   0.467700
    templated    aligned      0.725556   0.575008
                 constrained  0.000000   0.000000
                 raw          0.732963   0.580370
3   human        aligned      0.633333   0.400000
                 constrained  0.000000   0.000000
                 raw          0.522222   0.244444
    llama        aligned      0.710957   0.459299
                 constrained  0.000000  -0.059507
                 raw          0.680996   0.364789
    templated    aligned      0.811856   0.568738
                 constrained  0.000000  -0.044798
                 raw          0.791343   0.537818
5   human        aligned      0.687831   0.215488
                 constrained  0.000000  -0.127946
                 raw          0.896296   0.269360
    llama        aligned      0.663717   0.337104
                 constrained  0.000000  -0.088349
                 raw          0.674941   0.254612
    templated    aligned      0.827500   0.478308
                 constrained  0.000000  -0.069348
                 raw          0.742327   0.446131
7   llama        aligned      0.655197   0.286395
                 constrained  0.000000  -0.109264
                 raw          0.634964   0.183254
    templated    aligned      0.749200   0.408954
                 constrained  0.000000  -0.076415
                 raw          0.709646   0.391798
10  human        aligned      0.700926   0.241758
                 constrained  0.000000  -0.210623
                 raw          0.652219   0.170426

## Compare models across Datasets

In [23]:
datasets = [
    "DBpedia",
    # "UniProt",
    "Yago",
    "BTO",
    # "DNB",
]
grouped_dfs = {}
for dataset in datasets:
    model_compare = df_clean[df_clean["cfg_name"].str.strip() == dataset]
    model_compare = model_compare[
        ~model_compare["stage"].str.contains("constrained")
    ]
    grouped_model_compare = model_compare.groupby(
        [
            "n_nodes",
            "stage",
            "prompt_type",
            "model",
            # "generator",
        ]
    )[["f1_score", "normed_ged"]].mean()
    grouped_model_compare = grouped_model_compare.rename(
        columns=column_map
    ).rename_axis(
        [
            column_map["n_nodes"],
            column_map["stage"],
            column_map["prompt_type"],
            column_map["model"],
            # column_map["generator"],
        ],
        axis=0,
    )
    grouped_dfs[dataset] = grouped_model_compare
    with open(f"tables/compare_model_{dataset}.tex", "w") as f:
        f.write(
            grouped_model_compare.to_latex(
                float_format="%.2f",
                caption=f"Comparison of models for {dataset}",
                label=f"tab:compare_model_{dataset}",
                # sparsify=False
            )
        )
dataset_keys = list(grouped_dfs.keys())
merged_grouped: pd.DataFrame = grouped_dfs[dataset_keys[0]].copy()
merged_grouped.rename(
    columns={
        "$F_1$": f"$F_1$_{dataset_keys[0]}",
        "$GED_{s}$": f"$GED_{{s}}$_{dataset_keys[0]}",
    },
    inplace=True,
)
for dataset, grouped_df in list(grouped_dfs.items())[1:]:
    print(f"Merging {dataset} into the grouped DataFrame")
    merged_grouped = pd.merge(
        merged_grouped,
        grouped_dfs[dataset].rename(
            columns={
                "$F_1$": f"$F_1$_{dataset}",
                "$GED_{s}$": f"$GED_{{s}}$_{dataset}",
            }
        ),
        on=[
            "$k$",
            "Stage",
            "Prompt Type",
            "Version \\& Size",
            # "Query Origin",
        ],
        # suffixes=(None, f"_{dataset}"),
    )
rename_map = {f"$F_1$_{dataset}": f"{dataset} $F_1$" for dataset in datasets} | {
    f"$GED_{{s}}$_{dataset}": f"{dataset} $GED_{{s}}$" for dataset in datasets
}
merged_grouped.rename(
    columns=rename_map,
    inplace=True,
)
with open("tables/compare_model_merged.tex", "w") as f:
    f.write(
        merged_grouped.to_latex(
            float_format="%.2f",
            caption="Comparison of models for DBpedia and UniProt",
            label="tab:compare_model_merged",
        )
    )

Merging Yago into the grouped DataFrame
Merging BTO into the grouped DataFrame


In [24]:
grouped_dfs

{'DBpedia':                                             $F_1$  $GED_{s}$
 $k$ Stage   Prompt Type Version \& Size                     
 2   aligned one shot     3B              0.673129   0.517279
                         1 8B             0.536007   0.502700
             zero shot    3B              0.141753   0.257486
                         1 8B             0.137973   0.274296
     raw     one shot     3B              0.677211   0.519436
                         1 8B             0.509804   0.413624
             zero shot    3B              0.079210   0.050012
                         1 8B             0.076289   0.058861
 3   aligned one shot     3B              0.757154   0.509764
                         1 8B             0.684373   0.502706
             zero shot    3B              0.173630   0.250693
                         1 8B             0.294980   0.324773
     raw     one shot     3B              0.730459   0.444195
                         1 8B             0.652181   0.3829

In [25]:
merged_grouped

DBpedia $F_1$  DBpedia $GED_{s}$  \
$k$ Stage   Prompt Type Version \& Size                                     
2   aligned one shot     3B                   0.673129           0.517279   
                        1 8B                  0.536007           0.502700   
            zero shot    3B                   0.141753           0.257486   
                        1 8B                  0.137973           0.274296   
    raw     one shot     3B                   0.677211           0.519436   
                        1 8B                  0.509804           0.413624   
            zero shot    3B                   0.079210           0.050012   
                        1 8B                  0.076289           0.058861   
3   aligned one shot     3B                   0.757154           0.509764   
                        1 8B                  0.684373           0.502706   
            zero shot    3B                   0.173630           0.250693   
                        1 8B                  0.294980           0.324773   
    raw     one shot     3B                   0.730459           0.444195   
                        1 8B                  0.652181           0.382980   
            zero shot    3B                   0.071833           0.011952   
                        1 8B                  0.154414           0.045876   
5   aligned one shot     3B                   0.738977           0.399842   
                        1 8B                  0.701620           0.454505   
            zero shot    3B                   0.257825           0.245371   
                        1 8B                  0.470095           0.383946   
    raw     one shot     3B                   0.709093           0.342413   
                        1 8B                  0.645177           0.325454   
            zero shot    3B                   0.081827          -0.010295   
                        1 8B                  0.220677           0.035187   
7   aligned one shot     3B                   0.696976           0.340866   
                        1 8B                  0.709277           0.422026   
            zero shot    3B                   0.316217           0.241201   
                        1 8B                  0.536495           0.395057   
    raw     one shot     3B                   0.668156           0.275940   
                        1 8B                  0.659169           0.297520   
            zero shot    3B                   0.100379          -0.004505   
                        1 8B                  0.234857           0.011035   

                                         Yago $F_1$  Yago $GED_{s}$  \
$k$ Stage   Prompt Type Version \& Size                               
2   aligned one shot     3B                0.645791        0.542577   
                        1 8B               0.631633        0.483722   
            zero shot    3B                0.114478        0.188889   
                        1 8B               0.153667        0.210333   
    raw     one shot     3B                0.432323        0.253367   
                        1 8B               0.487415        0.295918   
            zero shot    3B                0.061279        0.035690   
                        1 8B               0.035000        0.023333   
3   aligned one shot     3B                0.641252        0.517628   
                        1 8B               0.673890        0.554303   
            zero shot    3B                0.224242        0.198219   
                        1 8B               0.180203        0.167536   
    raw     one shot     3B                0.486169        0.297980   
                        1 8B               0.530177        0.290944   
            zero shot    3B                0.089773        0.026353   
                        1 8B               0.027501        0.011142   
5   aligned one shot     3B                0.675311        0.473282   
                        1 8B               0.687070        0.501405   
  

In [19]:
df_clean["cfg_name"].str.strip() == "DBpedia"

0        True
1        True
2        True
3        True
4        True
        ...  
1129    False
1130    False
1131    False
1132    False
1133    False
Name: cfg_name, Length: 21405, dtype: bool